In [1]:
import numpy as np
import cv2
import onnx
import onnxruntime as ort
from PIL import Image
from pathlib import Path
# import vai_q_onnx

import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets import CIFAR10

import onnx
import onnxruntime as ort
from onnxruntime.quantization import CalibrationDataReader, QuantType, QuantFormat, CalibrationMethod, quantize_static

ModuleNotFoundError: No module named 'cv2'

In [2]:
import tarfile
import urllib.request
from resnet_utils import get_directories

_, models_dir, data_dir, _ = get_directories()
data_download_path_python = data_dir / "cifar-10-python.tar.gz"
data_download_path_bin = data_dir / "cifar-10-binary.tar.gz"
if not data_download_path_python.exists() or not data_download_path_bin.exists():
    urllib.request.urlretrieve("https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", data_download_path_python)
    urllib.request.urlretrieve("https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz", data_download_path_bin)
    file_python = tarfile.open(data_download_path_python)
    file_python.extractall(data_dir)
    file_python.close()
    file_bin = tarfile.open(data_download_path_bin)
    file_bin.extractall(data_dir)
    file_bin.close()

In [3]:
class CIFAR10DataSet:
    def __init__(
        self,
        data_dir,
        **kwargs,
    ):
        super().__init__()
        self.train_path = data_dir
        self.vld_path = data_dir
        self.setup("fit")

    def setup(self, stage: str):
        transform = transforms.Compose(
            [transforms.Pad(4), transforms.RandomHorizontalFlip(), transforms.RandomCrop(32), transforms.ToTensor()]
        )
        self.train_dataset = CIFAR10(root=self.train_path, train=True, transform=transform, download=False)
        self.val_dataset = CIFAR10(root=self.vld_path, train=True, transform=transform, download=False)


class PytorchResNetDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        sample = self.dataset[index]
        input_data = sample[0]
        label = sample[1]
        return input_data, label


class ResnetCalibrationDataReader(CalibrationDataReader):
    def __init__(self, data_dir: str, batch_size: int = 16):
        super().__init__()
        cifar10_dataset = CIFAR10DataSet(data_dir)
        _, val_set = torch.utils.data.random_split(cifar10_dataset.val_dataset, [49000, 1000])
        self.iterator = iter(DataLoader(PytorchResNetDataset(val_set), batch_size=batch_size, drop_last=True))

    def get_next(self) -> dict:
        try:
            images, labels = next(self.iterator)
            return {"input": images.numpy()}
        except Exception:
            return None

In [4]:
input_model_dir = 'models/onnx'
quant_model_dir = 'models/quant'

calibration_dataset_path = "data/"

import os
models = os.listdir(input_model_dir)
quantized_models = os.listdir(quant_model_dir)
for model in models:
    model_name = model.split('.')[0]
    print(model_name)

    input_model_path = os.path.join(input_model_dir, model_name + '.onnx')
    output_model_path = os.path.join(quant_model_dir, model_name + '.U8S8.onnx')

    dr = ResnetCalibrationDataReader(calibration_dataset_path, batch_size=128)

    vai_q_onnx.quantize_static(
        input_model_path,
        output_model_path,
        dr,
        quant_format=vai_q_onnx.QuantFormat.QDQ,
        calibrate_method=vai_q_onnx.PowerOfTwoMethod.MinMSE,
        activation_type=vai_q_onnx.QuantType.QUInt8,
        weight_type=vai_q_onnx.QuantType.QInt8,
        enable_dpu=True, 
        extra_options={'ActivationSymmetric': True} 
    )

    print('Calibrated and quantized model saved at:', output_model_path)

ResNet10_1111_4


INFO:vai_q_onnx.quant_utils:The input ONNX model models/onnx\ResNet10_1111_4.onnx can create InferenceSession successfully
INFO:vai_q_onnx.quant_utils:Obtained calibration data with 7 iters


[VAI_Q_ONNX_INFO]: Time information:
2024-04-15 23:09:54.501085
[VAI_Q_ONNX_INFO]: OS and CPU information:
                                        system --- Windows
                                          node --- GEEKOM-AMD
                                       release --- 10
                                       version --- 10.0.22631
                                       machine --- AMD64
                                     processor --- AMD64 Family 25 Model 116 Stepping 1, AuthenticAMD
[VAI_Q_ONNX_INFO]: Tools version information:
                                        python --- 3.9.19
                                          onnx --- 1.16.0
                                   onnxruntime --- 1.15.1
                                    vai_q_onnx --- 1.16.0+69bc4f2
[VAI_Q_ONNX_INFO]: Quantized Configuration information:
                                   model_input --- models/onnx\ResNet10_1111_4.onnx
                                  model_output --- models/quant\ResNet1

INFO:vai_q_onnx.quant_utils:The input ONNX model models/onnx\ResNet10_1111_4.onnx can run inference successfully
INFO:vai_q_onnx.quantize:Removed initializers from input
INFO:vai_q_onnx.quantize:Loading model...
INFO:vai_q_onnx.quantize:enable_ipu_cnn is True, optimize the model for better hardware compatibility.
INFO:vai_q_onnx.quantize:Start calibration...
INFO:vai_q_onnx.quantize:Start collecting data, runtime depends on your model size and the number of calibration dataset.
INFO:vai_q_onnx.calibrate:Finding optimal threshold for each tensor using PowerOfTwoMethod.MinMSE algorithm ...
INFO:vai_q_onnx.calibrate:Use all calibration data to calculate min mse
Computing range: 100%|██████████| 30/30 [00:05<00:00,  5.93tensor/s]
INFO:vai_q_onnx.qdq_quantizer:Remove QuantizeLinear & DequantizeLinear on certain operations(such as conv-relu).
INFO:vai_q_onnx.simulate_dpu:Rescale AveragePool /AveragePool with factor 1.0 to simulate DPU behavior.


┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Op Type              ┃ Float Model                            ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Conv                 │ 13                                     │
│ Relu                 │ 9                                      │
│ Add                  │ 4                                      │
│ AveragePool          │ 1                                      │
│ Shape                │ 1                                      │
│ Constant             │ 3                                      │
│ Gather               │ 1                                      │
│ Unsqueeze            │ 1                                      │
│ Concat               │ 1                                      │
│ Reshape              │ 1                                      │
│ Gemm                 │ 1                                      │
├──────────────────────┼────────────────────────────────────────┤
│ Quantized model path │ models/quant\ResNet10_1111_4.U8S8.onnx │
└──────────────────────┴────────────────────────────────────────┘

...

INFO:vai_q_onnx.quant_utils:The input ONNX model models/onnx\ResNet34_3463_8.onnx can create InferenceSession successfully
INFO:vai_q_onnx.quant_utils:Obtained calibration data with 7 iters


[VAI_Q_ONNX_INFO]: Time information:
2024-04-15 23:15:13.131203
[VAI_Q_ONNX_INFO]: OS and CPU information:
                                        system --- Windows
                                          node --- GEEKOM-AMD
                                       release --- 10
                                       version --- 10.0.22631
                                       machine --- AMD64
                                     processor --- AMD64 Family 25 Model 116 Stepping 1, AuthenticAMD
[VAI_Q_ONNX_INFO]: Tools version information:
                                        python --- 3.9.19
                                          onnx --- 1.16.0
                                   onnxruntime --- 1.15.1
                                    vai_q_onnx --- 1.16.0+69bc4f2
[VAI_Q_ONNX_INFO]: Quantized Configuration information:
                                   model_input --- models/onnx\ResNet34_3463_8.onnx
                                  model_output --- models/quant\ResNet3

INFO:vai_q_onnx.quant_utils:The input ONNX model models/onnx\ResNet34_3463_8.onnx can run inference successfully
INFO:vai_q_onnx.quantize:Removed initializers from input
INFO:vai_q_onnx.quantize:Loading model...
INFO:vai_q_onnx.quantize:enable_ipu_cnn is True, optimize the model for better hardware compatibility.
INFO:vai_q_onnx.quantize:Start calibration...
INFO:vai_q_onnx.quantize:Start collecting data, runtime depends on your model size and the number of calibration dataset.
INFO:vai_q_onnx.calibrate:Finding optimal threshold for each tensor using PowerOfTwoMethod.MinMSE algorithm ...
INFO:vai_q_onnx.calibrate:Use all calibration data to calculate min mse
Computing range: 100%|██████████| 90/90 [00:26<00:00,  3.36tensor/s]
INFO:vai_q_onnx.qdq_quantizer:Remove QuantizeLinear & DequantizeLinear on certain operations(such as conv-relu).
INFO:vai_q_onnx.simulate_dpu:Rescale AveragePool /AveragePool with factor 1.0 to simulate DPU behavior.


┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Op Type              ┃ Float Model                            ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Conv                 │ 37                                     │
│ Relu                 │ 33                                     │
│ Add                  │ 16                                     │
│ AveragePool          │ 1                                      │
│ Shape                │ 1                                      │
│ Constant             │ 3                                      │
│ Gather               │ 1                                      │
│ Unsqueeze            │ 1                                      │
│ Concat               │ 1                                      │
│ Reshape              │ 1                                      │
│ Gemm                 │ 1                                      │
├──────────────────────┼────────────────────────────────────────┤
│ Quantized model path │ models/quant\ResNet34_3463_8.U8S8.onnx │
└──────────────────────┴────────────────────────────────────────┘

Calibrated and quantized model saved at: models/quant\ResNet34_3463_8.U8S8.onnx
